In [ ]:
pip install xlsxwriter

In [ ]:
pip install geopandas

In [ ]:
pip install contextily

In [ ]:
pip install --upgrade pygbif

In [ ]:
pip install xarray netCDF4 matplotlib

In [ ]:
pip install xarray netCDF4 matplotlib

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# ADDING FEATURE - SPECIES TYPE - FINAL PEICE OF CODE
from pygbif import species

reptile_genera = [
    "Lophognathus", "Liolaemus", "Saproscincus", "Agama", "Plestiodon",
    "Philothamnus", "Sanzinia", "Ctenophorus", "Morethia", "Trimeresurus",
    "Atractus", "Phoenicolacerta", "Causus", "Contia", "Elgaria",
    "Chalcides", "Liodytes", "Chironius", "Tropidurus", "Dipsas",
    "Gallotia", "Pachydactylus", "Vipera", "Anguis", "Oligodon",
    "Pedioplanis", "Sceloporus", "Urosaurus", "Phrynosoma", "Bronchocela",
    "Alligator", "Gambelia", "Cacophis", "Boa", "Trioceros",
    "Pseudechis", "Pseudaspis", "Helicops", "Ctenotus", "Trachylepis",
    "Tiliqua", "Lycodon", "Malayemys", "Elaphe", "Crotaphytus",
    "Pogona", "Nerodia", "Nebulifera", "Python", "Lacerta",
    "Eutropis", "Lampropeltis", "Masticophis", "Afrogecko", "Naja",
    "Takydromus", "Amphisbaena", "Natrix", "Dispholidus", "Holcosus",
    "Furcifer", "Lycophidion", "Cyrtodactylus", "Chamaeleo", "Lamprophis",
    "Coelognathus", "Gambelia", "Lycodonomorphus"
]
reptile_genus_to_group = {genus: "Reptile" for genus in reptile_genera}

order_to_group = {
    'Squamata': 'Reptile', 'Testudines': 'Reptile', 'Crocodylia': 'Reptile',
    'Anura': 'Amphibian', 'Caudata': 'Amphibian',
    'Passeriformes': 'Bird', 'Accipitriformes': 'Bird', 'Columbiformes': 'Bird',
    'Psittaciformes': 'Bird', 'Strigiformes': 'Bird', 'Falconiformes': 'Bird',
    'Carnivora': 'Mammal', 'Primates': 'Mammal', 'Rodentia': 'Mammal', 'Chiroptera': 'Mammal'
}

class_to_group = {
    'Amphibia': 'Amphibian',
    'Aves': 'Bird',
    'Mammalia': 'Mammal'
}

def classify_species(scientific_name):
    try:
        genus = scientific_name.split()[0]

        if genus in reptile_genus_to_group:
            return "Reptile"

        res = species.name_backbone(scientific_name, strict=False)
        cls = res.get('class')
        order = res.get('order')

        if cls in class_to_group:
            return class_to_group[cls]
        if order in order_to_group:
            return order_to_group[order]

        res_genus = species.name_backbone(genus, strict=False)
        cls_genus = res_genus.get('class')
        order_genus = res_genus.get('order')
        if cls_genus in class_to_group:
            return class_to_group[cls_genus]
        if order_genus in order_to_group:
            return order_to_group[order_genus]

        return "Unknown"

    except Exception as e:
        return "Unknown"

In [ ]:
df1 = pd.read_excel("./dataprep/species_train.xlsx", sheet_name='taxon_names')
df1.rename(columns={0: 'taxon_names'}, inplace=True)
df1['taxon_names'] = df1['taxon_names'].astype(str).str.strip()
df1['Group'] = df1['taxon_names'].apply(classify_species)
with pd.ExcelWriter("./dataprep/species_train.xlsx", engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df1.to_excel(writer, sheet_name='taxon_names', index=False)

In [ ]:
print(df1.head())

In [ ]:
# ADDING FEATURE - SPECIES TYPE - FINAL PEICE OF CODE
import pandas as pd


excel_file = "./dataprep/species_train.xlsx"


df_locs = pd.read_excel(excel_file, sheet_name='train_locs')
df_ids = pd.read_excel(excel_file, sheet_name='train_ids')
df_taxon_ids = pd.read_excel(excel_file, sheet_name='taxon_ids')
df_taxon_names = pd.read_excel(excel_file, sheet_name='taxon_names')
df_locs = df_locs.rename(columns={0:'latitude', 1:'longitude'})

df_taxon_full = pd.concat([df_taxon_ids.reset_index(drop=True), df_taxon_names.reset_index(drop=True)], axis=1)
df_merged = pd.concat([df_locs[['latitude','longitude']], df_ids[['train_ids']]], axis=1)
df_merged = df_merged[df_merged['train_ids'].isin(df_taxon_full['taxon_ids'])]
df_final = df_merged.merge(
    df_taxon_full,
    left_on='train_ids',
    right_on='taxon_ids',
    how='left'
)

df_final = df_final.rename(columns={'taxon_names':'taxon_name', 'Group':'type'})\
                   [['latitude', 'longitude', 'train_ids', 'taxon_name', 'type']]

print(df_final.head())

In [ ]:
with pd.ExcelWriter(excel_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_final.to_excel(writer, sheet_name='merged_data', index=False)

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np


ds = xr.open_dataset("./dataprep/C3S-LC-L4-LCCS-Map-300m-P1Y-2022-v2.1.1.nc")

lc = ds['lccs_class'].isel(time=0)

lats = ds['lat'].values
lons = ds['lon'].values

points_df = pd.DataFrame({
    'Latitude': [-18.28672791],
    'Longitude': [143.4812469]
})

def nearest_index(array, value):
    return (np.abs(array - value)).argmin()


land_cover_values = []
for idx, row in points_df.iterrows():
    lat_idx = nearest_index(lats, row['Latitude'])
    lon_idx = nearest_index(lons, row['Longitude'])

    lc_value = lc.values[lat_idx, lon_idx]
    land_cover_values.append(lc_value)

points_df['Land_Cover_Code'] = land_cover_values
print(points_df)

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
excel_path = "./dataprep/species_test.xlsx"
df = pd.read_excel(excel_path,sheet_name='test_locs')
df = df.rename(columns={0:'latitude', 1:'longitude'})
print(list(df))

In [ ]:
# ADDING FEATURE LAND COVER TYPE - FINAL CODE USED (for both train and test data)
import xarray as xr
import pandas as pd
import numpy as np

excel_path = "./dataprep/species_test.xlsx"
df = pd.read_excel(excel_path,sheet_name='test_locs')
df = df.rename(columns={0:'latitude', 1:'longitude'}) #Only for test dataset
lat_col = 'latitude'
lon_col = 'longitude'

nc_path = "./dataprep/C3S-LC-L4-LCCS-Map-300m-P1Y-2022-v2.1.1.nc"
ds = xr.open_dataset(nc_path)

lc = ds['lccs_class'].isel(time=0)

lats = ds['lat'].values
lons = ds['lon'].values

def nearest_index(array, value):
    return (np.abs(array - value)).argmin()

land_cover_map = {
    0: {"name": "No Data", "category": "Unknown"},
    10: {"name": "Cropland, rainfed", "category": "Agriculture"},
    11: {"name": "Herbaceous cover", "category": "Agriculture"},
    12: {"name": "Tree or shrub cover", "category": "Agriculture"},
    20: {"name": "Cropland, irrigated or post-flooding", "category": "Agriculture"},
    30: {"name": "Mosaic cropland (>50%) / natural vegetation (<50%)", "category": "Mixed vegetation"},
    40: {"name": "Mosaic natural vegetation (>50%) / cropland (<50%)", "category": "Mixed vegetation"},
    50: {"name": "Tree cover, broadleaved, evergreen, closed to open (>15%)", "category": "Forest"},
    60: {"name": "Tree cover, broadleaved, deciduous, closed to open (>15%)", "category": "Forest"},
    61: {"name": "Tree cover, broadleaved, deciduous, closed to open (>40%)", "category": "Forest"},
    62: {"name": "Tree cover, broadleaved, deciduous, open (15–40%)", "category": "Forest"},
    70: {"name": "Tree cover, needleleaved, evergreen, closed to open (>15%)", "category": "Forest"},
    71: {"name": "Tree cover, needleleaved, evergreen, closed to open (>40%)", "category": "Forest"},
    72: {"name": "Tree cover, needleleaved, evergreen, open (15–40%)", "category": "Forest"},
    80: {"name": "Tree cover, needleleaved, deciduous, closed to open (>15%)", "category": "Forest"},
    81: {"name": "Tree cover, needleleaved, deciduous, closed to open (>40%)", "category": "Forest"},
    82: {"name": "Tree cover, needleleaved, deciduous, open (15–40%)", "category": "Forest"},
    90: {"name": "Tree cover, mixed leaf type (broadleaved + needleleaved)", "category": "Forest"},
    100: {"name": "Mosaic tree and shrub (>50%) / herbaceous cover (<50%)", "category": "Mixed vegetation"},
    110: {"name": "Mosaic herbaceous cover (>50%) / tree and shrub (<50%)", "category": "Mixed vegetation"},
    120: {"name": "Shrubland", "category": "Shrubland"},
    121: {"name": "Evergreen shrubland", "category": "Shrubland"},
    122: {"name": "Deciduous shrubland", "category": "Shrubland"},
    130: {"name": "Grassland", "category": "Grassland"},
    140: {"name": "Lichens and mosses", "category": "Tundra"},
    150: {"name": "Sparse vegetation (tree/shrub/herbaceous cover <15%)", "category": "Semi-arid"},
    151: {"name": "Sparse tree (<15%)", "category": "Semi-arid"},
    152: {"name": "Sparse shrub (<15%)", "category": "Semi-arid"},
    153: {"name": "Sparse herbaceous cover (<15%)", "category": "Semi-arid"},
    160: {"name": "Tree cover, flooded, fresh or brackish water", "category": "Wetlands"},
    170: {"name": "Tree cover, flooded, saline water", "category": "Wetlands"},
    180: {"name": "Shrub or herbaceous cover, flooded, fresh/saline/brackish water", "category": "Wetlands"},
    190: {"name": "Urban areas", "category": "Urban"},
    200: {"name": "Bare areas", "category": "Bare land"},
    201: {"name": "Consolidated bare areas", "category": "Bare land"},
    202: {"name": "Unconsolidated bare areas", "category": "Bare land"},
    210: {"name": "Water bodies", "category": "Water"},
    220: {"name": "Permanent snow & ice", "category": "Ice/Snow"}
}

land_cover_types = []

for idx, row in df.iterrows():
    lat = row[lat_col]
    lon = row[lon_col]

    lat_idx = nearest_index(lats, lat)
    lon_idx = nearest_index(lons, lon)

    lc_code = int(lc.values[lat_idx, lon_idx])

    info = land_cover_map.get(lc_code, {"name": "Unknown", "category": "Unknown"})

    land_cover_types.append(info["category"])

df["Land_Cover_Type"] = land_cover_types

with pd.ExcelWriter(excel_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df.to_excel(writer, sheet_name="test_locs", index=False)

print(df.head())

In [ ]:
pip install earthengine-api

In [ ]:
pip install geopy

In [ ]:
oceans = gpd.read_file("./dataprep/GOaS_v1_20211214/goas_v01.shp")
print(oceans.columns)

In [ ]:
pip install fiona

In [ ]:
import fiona
fiona.listlayers("./dataprep/gadm_410.gpkg")

In [ ]:
gadm = gpd.read_file("./dataprep/gadm_410.gpkg")
print(gadm.columns)

In [ ]:
# ADDING FEATURE REGION IN DATASET - FINAL PEICE OF CODE USED (for both train and test data)
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

excel_path = "./dataprep/species_test.xlsx"
sheet_name = "test_locs"

df = pd.read_excel(excel_path, sheet_name=sheet_name)
geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
gdf_points = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

gadm = gpd.read_file("./dataprep/gadm_410.gpkg")
gadm = gadm[['NAME_0', 'NAME_1', 'geometry']]

gdf_points = gdf_points.to_crs(gadm.crs)

gdf_points = gpd.sjoin(
    gdf_points,
    gadm,
    how="left",
    predicate="within",
    lsuffix='left',
    rsuffix='right'
)

candidates = gdf_points[gdf_points['NAME_1'].isna()].copy()
if 'index_right' in candidates.columns:
    candidates.drop(columns=['index_right'], inplace=True)

goas = gpd.read_file("./dataprep/GOaS_v1_20211214/goas_v01.shp")
marine_name_col = 'name'

candidates = candidates.to_crs(goas.crs)

marine_points = gpd.sjoin(
    candidates,
    goas[['geometry', marine_name_col]],
    how="left",
    predicate="within",
    lsuffix='left',
    rsuffix='right'
)

gdf_points['region'] = gdf_points['NAME_1']

gdf_points.loc[gdf_points.index.isin(marine_points.index), 'region'] = marine_points[marine_name_col].values

def polar_region(lat):
    if lat < -60:
        return "Ice/Snow"
    elif lat > 75:
        return "Ice/Snow"
    else:
        return None

mask_unknown = gdf_points['region'].isna() | (gdf_points['region']=="")
gdf_points.loc[mask_unknown, 'region'] = gdf_points.loc[mask_unknown, 'latitude'].apply(polar_region)

gdf_points['region'] = gdf_points['region'].fillna("Unknown")

output_cols = df.columns.tolist() + ['region']
final_df = gdf_points[output_cols]

from openpyxl import load_workbook

with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    final_df.to_excel(writer, sheet_name=sheet_name, index=False)


In [ ]:
# ADDING FEATURE - ANNUAL AVERAGE TEMPERATURE & PRECIPITATION - FINAL PEICE OF CODE USED (for both train and test data)
import os
import pandas as pd
import rasterio
from shapely.geometry import Point
import numpy as np

excel_path = "./dataprep/species_test.xlsx"
sheet_name = "test_locs"

df = pd.read_excel(excel_path, sheet_name=sheet_name)
geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]

tmin_folder = "./dataprep/wc2.1_cruts4.09_5m_tmin_2020-2024"
tmax_folder = "./dataprep/wc2.1_cruts4.09_5m_tmax_2020-2024"
prec_folder = "./dataprep/wc2.1_cruts4.09_5m_prec_2020-2024"

tmin_files = sorted([os.path.join(tmin_folder, f) for f in os.listdir(tmin_folder) if f.endswith(".tif")])
tmax_files = sorted([os.path.join(tmax_folder, f) for f in os.listdir(tmax_folder) if f.endswith(".tif")])
prec_files = sorted([os.path.join(prec_folder, f) for f in os.listdir(prec_folder) if f.endswith(".tif")])

def extract_raster_values(raster_files, points):
    values_all = []
    for f in raster_files:
        with rasterio.open(f) as src:
            vals = []
            for pt in points:
                try:
                    val = list(src.sample([(pt.x, pt.y)]))[0][0]
                    if np.isnan(val):
                        val = np.nan
                except:
                    val = np.nan
                vals.append(val)
            values_all.append(vals)
    return list(map(list, zip(*values_all)))

tmin_values = extract_raster_values(tmin_files, geometry)
tmax_values = extract_raster_values(tmax_files, geometry)
prec_values = extract_raster_values(prec_files, geometry)

def group_by_year(files):
    groups = {}
    for i, f in enumerate(files):
        year = os.path.basename(f).split("_")[-1].split("-")[0]
        groups.setdefault(year, []).append(i)
    return groups

tmin_groups = group_by_year(tmin_files)
tmax_groups = group_by_year(tmax_files)
prec_groups = group_by_year(prec_files)

for year in tmin_groups.keys():
    tavg_year = []
    prec_year = []

    for point_idx in range(len(df)):
        tmin_months = [tmin_values[point_idx][i] for i in tmin_groups[year]]
        tmax_months = [tmax_values[point_idx][i] for i in tmax_groups[year]]
        prec_months = [prec_values[point_idx][i] for i in prec_groups[year]]

        tavg_months = [(tmin + tmax) / 2 if not np.isnan(tmin) and not np.isnan(tmax) else np.nan
                       for tmin, tmax in zip(tmin_months, tmax_months)]

        tavg_year.append(np.nanmean(tavg_months))
        prec_year.append(np.nanmean(prec_months))

    df[f'tavg_{year}_mean'] = tavg_year
    df[f'prec_{year}_mean'] = prec_year

from openpyxl import load_workbook

with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df.to_excel(writer, sheet_name=sheet_name, index=False)


In [ ]:
# ADDING FEATURE - ELEVATION - FINAL PEICE OF CODE USED (for both train and test data)
import rasterio
import pandas as pd
from shapely.geometry import Point
import numpy as np
import os

#excel_path = "/Users/jayashreehariharan/Desktop/AML_Project/Species-2/species/Merged_train_data.xlsx"
#sheet_name = "merged_data"
excel_path = "./dataprep/species_test.xlsx"
sheet_name = "test_locs"

df = pd.read_excel(excel_path, sheet_name=sheet_name)
geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]

elev_path = "./dataprep/wc2.1_5m_elev.tif"

elevation_values = []
with rasterio.open(elev_path) as src:
    for pt in geometry:
        try:
            val = list(src.sample([(pt.x, pt.y)]))[0][0]
            if np.isnan(val):
                val = np.nan
        except:
            val = np.nan
        elevation_values.append(val)

df["elevation"] = elevation_values

from openpyxl import load_workbook

with pd.ExcelWriter(excel_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df.to_excel(writer, sheet_name=sheet_name, index=False)

## Feature Engineering

In [ ]:
#Feature Engineering - adding climate trend features - Train Dataset
import pandas as pd
import numpy as np

excel_path = "./dataprep/Merged_train_data.xlsx"
sheet_name = "merged_data"
df = pd.read_excel(excel_path, sheet_name=sheet_name)

tavg_cols = ['tavg_2020_mean','tavg_2021_mean','tavg_2022_mean','tavg_2023_mean','tavg_2024_mean']
prec_cols = ['prec_2020_mean','prec_2021_mean','prec_2022_mean','prec_2023_mean','prec_2024_mean']

years = np.array([2020, 2021, 2022, 2023, 2024])

tavg_values = df[tavg_cols].values

tavg_trend = np.polyfit(years, tavg_values.T, 1)[0]

def compute_slope(values, years):
    x = years
    x_mean = x.mean()
    x_diff2 = np.sum((x - x_mean)**2)
    slopes = np.sum((values - values.mean(axis=1, keepdims=True)) * (x - x_mean), axis=1) / x_diff2
    return slopes

df['tavg_trend'] = compute_slope(tavg_values, years)
df['prec_trend'] = compute_slope(df[prec_cols].values, years)

df['tavg_std'] = tavg_values.std(axis=1)
df['prec_std'] = df[prec_cols].values.std(axis=1)

df['tavg_max'] = tavg_values.max(axis=1)
df['tavg_min'] = tavg_values.min(axis=1)
df['tavg_range'] = df['tavg_max'] - df['tavg_min']

df['prec_max'] = df[prec_cols].values.max(axis=1)
df['prec_min'] = df[prec_cols].values.min(axis=1)
df['prec_range'] = df['prec_max'] - df['prec_min']

df['tavg_cv'] = df['tavg_std'] / df['tavg_mean'] if 'tavg_mean' in df.columns else df['tavg_std'] / tavg_values.mean(axis=1)
df['prec_cv'] = df['prec_std'] / df['prec_mean'] if 'prec_mean' in df.columns else df['prec_std'] / df[prec_cols].values.mean(axis=1)

with pd.ExcelWriter(excel_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
#Feature Engineering - adding climate trend features - Test Dataset
import pandas as pd
import numpy as np

excel_path = "./dataprep/species_test.xlsx"
sheet_name = "test_locs"
df = pd.read_excel(excel_path, sheet_name=sheet_name)

tavg_cols = ['tavg_2020_mean','tavg_2021_mean','tavg_2022_mean','tavg_2023_mean','tavg_2024_mean']
prec_cols = ['prec_2020_mean','prec_2021_mean','prec_2022_mean','prec_2023_mean','prec_2024_mean']

years = np.array([2020, 2021, 2022, 2023, 2024])

tavg_values = df[tavg_cols].values

tavg_trend = np.polyfit(years, tavg_values.T, 1)[0]

def compute_slope(values, years):
    x = years
    x_mean = x.mean()
    x_diff2 = np.sum((x - x_mean)**2)
    slopes = np.sum((values - values.mean(axis=1, keepdims=True)) * (x - x_mean), axis=1) / x_diff2
    return slopes

df['tavg_trend'] = compute_slope(tavg_values, years)
df['prec_trend'] = compute_slope(df[prec_cols].values, years)

df['tavg_std'] = tavg_values.std(axis=1)
df['prec_std'] = df[prec_cols].values.std(axis=1)

df['tavg_max'] = tavg_values.max(axis=1)
df['tavg_min'] = tavg_values.min(axis=1)
df['tavg_range'] = df['tavg_max'] - df['tavg_min']

df['prec_max'] = df[prec_cols].values.max(axis=1)
df['prec_min'] = df[prec_cols].values.min(axis=1)
df['prec_range'] = df['prec_max'] - df['prec_min']

df['tavg_cv'] = df['tavg_std'] / df['tavg_mean'] if 'tavg_mean' in df.columns else df['tavg_std'] / tavg_values.mean(axis=1)
df['prec_cv'] = df['prec_std'] / df['prec_mean'] if 'prec_mean' in df.columns else df['prec_std'] / df[prec_cols].values.mean(axis=1)

with pd.ExcelWriter(excel_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
#Feature Engineering - encoding spatial continuity - Train Dataset
import pandas as pd
import numpy as np
from scipy.stats import linregress

excel_path = "./dataprep/Merged_train_data.xlsx"
sheet_name = "merged_data"
df = pd.read_excel(excel_path, sheet_name=sheet_name)

df['lat_rad'] = np.radians(df['latitude'])
df['lon_rad'] = np.radians(df['longitude'])

df['lat_sin'] = np.sin(df['lat_rad'])
df['lat_cos'] = np.cos(df['lat_rad'])
df['lon_sin'] = np.sin(df['lon_rad'])
df['lon_cos'] = np.cos(df['lon_rad'])

df.drop(['lat_rad','lon_rad'], axis=1, inplace=True)

with pd.ExcelWriter(excel_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
#Feature Engineering - encoding spatial continuity - Test Dataset
import pandas as pd
import numpy as np
from scipy.stats import linregress

excel_path = "./dataprep/species_test.xlsx"
sheet_name = "test_locs"
df = pd.read_excel(excel_path, sheet_name=sheet_name)

df['lat_rad'] = np.radians(df['latitude'])
df['lon_rad'] = np.radians(df['longitude'])

df['lat_sin'] = np.sin(df['lat_rad'])
df['lat_cos'] = np.cos(df['lat_rad'])
df['lon_sin'] = np.sin(df['lon_rad'])
df['lon_cos'] = np.cos(df['lon_rad'])

df.drop(['lat_rad','lon_rad'], axis=1, inplace=True)

with pd.ExcelWriter(excel_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
#Adding all species taxon_id's found at a certain latitude and longitude
import pandas as pd
import ast
from tqdm import tqdm

file_path = "./dataprep/species_test.xlsx"
output_file = "./dataprep/species_test_with_species.xlsx"

sheet_locs = 'test_locs'
sheet_indices = 'test_pos_inds'
sheet_taxon_ids = 'taxon_ids'

df_locs = pd.read_excel(file_path, sheet_name=sheet_locs)
df_indices = pd.read_excel(file_path, sheet_name=sheet_indices)
df_species_ids = pd.read_excel(file_path, sheet_name=sheet_taxon_ids)

def fix_list_string(s):
    s = str(s).strip()
    if not s.startswith('['):
        s = '[' + s
    if s.endswith(','):
        s = s[:-1] + ']'
    elif not s.endswith(']'):
        s = s + ']'
    return s

indices_lists = []
for i, row in tqdm(df_indices.iterrows(), total=df_indices.shape[0], desc="Processing indices"):
    cell = row[0]
    fixed = fix_list_string(cell)
    try:
        indices_list = ast.literal_eval(fixed)
    except Exception as e:
        print(f"Error parsing row {i}: {e}")
        indices_list = []
    indices_lists.append(indices_list)

species_idx_pairs = [
    (sp_id, [x-1 for x in idx_list if pd.notna(x)])
    for sp_id, idx_list in zip(df_species_ids.iloc[:,0], indices_lists)
]

df_locs['species'] = [[] for _ in range(len(df_locs))]

for sp_id, idx_list in tqdm(species_idx_pairs, desc="Mapping species to locations"):
    for idx in idx_list:
        if 0 <= idx < len(df_locs):
            df_locs.at[idx, 'species'].append(sp_id)

with pd.ExcelWriter(file_path, engine="openpyxl") as writer:
    df_locs.to_excel(writer, sheet_name=sheet_locs, index=False)

In [ ]:
# Removing all data in test dataset based on latitudes and longitudes combo not having any species
import pandas as pd
excel_path = "./dataprep/test_data_locs_with_species_only.xlsx"
sheet_name = "test_locs"
df = pd.read_excel(excel_path, sheet_name=sheet_name)

df = df[df.species != '[]']

with pd.ExcelWriter(excel_path, engine="openpyxl") as writer:
    df.to_excel(writer, sheet_name="clean_test", index=False)

## Spatial Extrapolation

In [ ]:
#SPATIAL EXTRAPOLATION FOR TEMPERATURE AND PRECIPITATION - TRAIN DATA
import pandas as pd
import numpy as np

excel_path = "./dataprep/Merged_train_data.xlsx"
sheet_name = "merged_data"
df = pd.read_excel(excel_path, sheet_name=sheet_name)

cols_to_fill = [
    'tavg_2020_mean','prec_2020_mean',
    'tavg_2021_mean','prec_2021_mean',
    'tavg_2022_mean','prec_2022_mean',
    'tavg_2023_mean','prec_2023_mean',
    'tavg_2024_mean','prec_2024_mean'
]

valid = df.dropna(subset=cols_to_fill, how='all').reset_index(drop=True)
missing = df[df[cols_to_fill].isna().any(axis=1)].reset_index()

def haversine_vectorized(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1[:, None]
    dlon = lon2 - lon1[:, None]
    a = np.sin(dlat/2)**2 + np.cos(lat1)[:, None] * np.cos(lat2) * np.sin(dlon/2)**2
    a = np.clip(a, 0, 1)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

dist_matrix = haversine_vectorized(
    missing['latitude'].values,
    missing['longitude'].values,
    valid['latitude'].values,
    valid['longitude'].values
)

nearest_indices = np.argmin(dist_matrix, axis=1)

for idx_missing, idx_valid in zip(missing['index'], nearest_indices):
    for col in cols_to_fill:
        if pd.isna(df.at[idx_missing, col]):
            df.at[idx_missing, col] = valid.at[idx_valid, col]

with pd.ExcelWriter(excel_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
#SPATIAL EXTRAPOLATION FOR TEMPERATURE AND PRECIPITATION - TEST DATA
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree

excel_path = "./dataprep/species_test.xlsx"
sheet_name = "test_locs"
df = pd.read_excel(excel_path, sheet_name=sheet_name)

cols_to_fill = [
    'tavg_2020_mean','prec_2020_mean',
    'tavg_2021_mean','prec_2021_mean',
    'tavg_2022_mean','prec_2022_mean',
    'tavg_2023_mean','prec_2023_mean',
    'tavg_2024_mean','prec_2024_mean'
]

valid = df.dropna(subset=cols_to_fill, how='all').reset_index(drop=True)
missing = df[df[cols_to_fill].isna().any(axis=1)].reset_index()

def latlon_to_xyz(lat, lon):
    lat_rad = np.radians(lat)
    lon_rad = np.radians(lon)
    x = np.cos(lat_rad) * np.cos(lon_rad)
    y = np.cos(lat_rad) * np.sin(lon_rad)
    z = np.sin(lat_rad)
    return np.column_stack((x, y, z))

valid_xyz = latlon_to_xyz(valid['latitude'].values, valid['longitude'].values)
missing_xyz = latlon_to_xyz(missing['latitude'].values, missing['longitude'].values)

tree = cKDTree(valid_xyz)

_, nearest_idx = tree.query(missing_xyz, k=1)

for idx_missing, idx_valid in zip(missing['index'], nearest_idx):
    for col in cols_to_fill:
        if pd.isna(df.at[idx_missing, col]):
            df.at[idx_missing, col] = valid.at[idx_valid, col]

with pd.ExcelWriter(excel_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df.to_excel(writer, sheet_name=sheet_name, index=False)

print("Missing values filled")